In [ ]:
-- Enable Polybase
-- exec sp_configure @configname = 'polybase enabled', @configvalue = 1;
-- RECONFIGURE with override;

-- exec sp_configure 'polybase enabled';

select SERVERPROPERTY('IsPolyBaseInstalled')

In [ ]:
-- database in context
use COE
go

In [ ]:
-- create external data source for HDFS inside SQLBDC
--

if not exists(select * from sys.external_data_sources where name = 'SqlStoragePool')
    create external data source SqlStoragePool
    with (location = 'sqlhdfs://controller-svc/default')

In [ ]:
-- Create file format for parquet file with appropriate properties
--
if not exists(select * from sys.external_file_formats where name = 'parquet_file')
    create external file FORMAT parquet_file
    with (

        format_type = PARQUET
    )

In [ ]:
-- create external table
-- ingesting data from HDFS or Hive to SqlStoragePool

if exists(select * from sys.external_tables where name = 'scm_h_po_demo_data')
    drop EXTERNAL TABLE [COE].[demo].[scm_h_po_demo_data]
begin 
    create EXTERNAL TABLE [COE].[demo].[scm_h_po_demo_data]
    (        
        "HPO_PO_Number" varchar(30) not null,
        "HPO_Job_Code" varchar(8) not null
    )

    with 
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/user/hive/warehouse/po_data_sample',
        FILE_FORMAT = parquet_file

    )
end;


In [ ]:
select * from demo.scm_h_po_demo_data